## Iris Flower classification


In [2]:
from __future__ import absolute_import , division , print_function , unicode_literals

import tensorflow as tf
import pandas as pd

# Check TensorFlow version
print(tf.__version__)

# Create a simple tensor
# tensor = tf.constant([[1, 2], [3, 4]])
# print(tensor)
# tensor_reshape = tf.reshape(tensor,[4,1])
# print(tensor_reshape)


2.19.0


In [3]:
CSV_COLUMN_NAMES = ['SepalLength' , 'SepalWidth' , 'PetalLength' , 'PetalWidth' , 'Species']
SPECIES = ['Setosa'  , 'Versicolor' , 'Verginica']
#Constants to help us later on

In [4]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv" , "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"
)
test_path = tf.keras.utils.get_file(
    "iris_test.csv" , "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv"
)

train = pd.read_csv(train_path , names = CSV_COLUMN_NAMES , header =0)
test = pd.read_csv(test_path , names = CSV_COLUMN_NAMES , header = 0)

2194/2194 ━━━━━━━━━━━━━━━━━━━━ 0s 27us/step
573/573 ━━━━━━━━━━━━━━━━━━━━ 0s 8us/step


In [5]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [ ]:
train_y = train.pop('Species')
test_y = test.pop('Species')
# train_y.head


## Tensor Flow data set

In [ ]:


def create_dataset(features, labels, training=True, batch_size=256, num_epochs=None):
    """
    Creates a TensorFlow Dataset from features and labels.
    
    Args:
        features (dict): Dictionary of feature tensors or numpy arrays.
        labels: Label tensor or numpy array.
        training (bool): Whether to shuffle and repeat the dataset (default: True).
        batch_size (int): Size of each batch (default: 256).
        num_epochs (int, optional): Number of epochs to repeat the dataset (default: None, infinite if training).
    
    Returns:
        tf.data.Dataset: A batched and optionally shuffled/repeated dataset.
    """
    # Convert features dictionary and labels to a Dataset
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    
    # Shuffle and repeat if in training mode
    if training:
        dataset = dataset.shuffle(buffer_size=1000)  # Shuffle with a buffer of 1000 samples
        if num_epochs is not None:
            dataset = dataset.repeat(num_epochs)  # Repeat for specified epochs
        else:
            dataset = dataset.repeat()  # Infinite repeat if no epochs specified
    
    # Batch the dataset
    dataset = dataset.batch(batch_size)
    
    # Prefetch to improve performance
    dataset = dataset.prefetch(tf.data.AUTOTUNE)  # Allows background processing of data
    
    return dataset

## Feature COlumns

In [ ]:
X_train = train.values
print(X_train)



In [ ]:
# Optionally, normalize the data (example with tf.keras.layers.Normalization)
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(X_train)  # Compute mean and variance for normalization
# Build a Keras model with the normalized input
model = tf.keras.Sequential([
    normalizer,  # Apply normalization layer
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



# Train the model (using tf.data.Dataset for batching, optional)
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, train_y)).batch(32)

model.fit(train_dataset, epochs=10, steps_per_epoch=len(X_train) // 32)

KeyError: 'label'